In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [4]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2018-08-01 17:15 /data/wiki/en_articles_part


In [5]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

0%however	1
0&\mathrm{if	1
0(8)320-1234	1
0)).(1	2
0,03	1
0,1,...,n	1
0,1,0	1
0,1,\dots,n	1
0,5	1
0,50	1


rm: `wordcount_result_1536009993654042': No such file or directory
18/09/03 21:26:37 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/03 21:26:37 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/09/03 21:26:38 INFO mapred.FileInputFormat: Total input files to process : 1
18/09/03 21:26:38 INFO mapreduce.JobSubmitter: number of splits:2
18/09/03 21:26:38 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1535999871863_0005
18/09/03 21:26:38 INFO impl.YarnClientImpl: Submitted application application_1535999871863_0005
18/09/03 21:26:38 INFO mapreduce.Job: The url to track the job: http://f174b2b693e6:8088/proxy/application_1535999871863_0005/
18/09/03 21:26:38 INFO mapreduce.Job: Running job: job_1535999871863_0005
18/09/03 21:26:44 INFO mapreduce.Job: Job job_1535999871863_0005 running in uber mode : false
18/09/03 21:26:44 INFO mapreduce.Job:  map 0% reduce 0%
18/09/03 21:27:01 INFO mapreduce.Job:  map 31% reduce 0%
18/09/03 21:2